In [2]:
pip install simpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import simpy
import random
import statistics

# --- Configuration Parameters ---
# FAB_STEPS_CONFIG defines each manufacturing step:
#   - capacity: Number of machines available for this step.
#   - process_time_mean: Average time (in simulation units) a wafer spends at this step.
#   - process_time_std: Standard deviation for process time, adding randomness.
FAB_STEPS_CONFIG = {
    "Lithography": {"capacity": 2, "process_time_mean": 10, "process_time_std": 2},
    "Etch": {"capacity": 3, "process_time_mean": 8, "process_time_std": 1},
    "Deposition": {"capacity": 2, "process_time_mean": 12, "process_time_std": 3},
    "IonImplant": {"capacity": 1, "process_time_mean": 15, "process_time_std": 2}, # This step has low capacity and high process time, potentially a bottleneck.
    "Metrology": {"capacity": 4, "process_time_mean": 5, "process_time_std": 1},
}

WAFER_ARRIVAL_INTERVAL = 5 # Mean time (in simulation units) between new wafer arrivals.
SIMULATION_TIME = 200 # Total duration (in simulation units) for which the simulation will run.

# --- Global Data Collections for Analysis ---
# These lists and dictionaries will store performance data collected during the simulation.
wafer_cycle_times = [] # Stores total time each wafer spends in the system from arrival to completion.
step_queue_times = {step: [] for step in FAB_STEPS_CONFIG} # Stores waiting times for wafers at the queue of each step.
step_process_times = {step: [] for step in FAB_STEPS_CONFIG} # Stores actual processing times for wafers at each step.
# Stores snapshots of resource utilization: (time, busy_ratio) where busy_ratio is (busy_machines / total_capacity).
resource_utilization_data = {step: [] for step in FAB_STEPS_CONFIG}


# --- Wafer Class ---
class Wafer:
    """
    Represents a single wafer moving through the semiconductor fabrication process.
    Each wafer is an entity that experiences events (arrival, entering queue,
    starting process, finishing process, completing all steps).
    """
    def __init__(self, env, name):
        """
        Initializes a new Wafer object.
        :param env: The SimPy environment instance.
        :param name: A unique identifier for the wafer (e.g., "Wafer_1").
        """
        self.env = env # Reference to the simulation environment
        self.name = name # Unique name for this wafer
        self.arrival_time = env.now # Records the simulation time when the wafer is created
        self.start_times = {} # Dictionary to store the start time of processing for each step
        self.end_times = {}   # Dictionary to store the end time of processing for each step
        self.queue_enter_times = {} # Dictionary to store the time when the wafer entered the queue for each step

    def __repr__(self):
        """
        Provides a string representation of the Wafer object, useful for debugging.
        """
        return f"Wafer(name={self.name}, arrival={self.arrival_time:.2f})"


# --- Fabrication Step Process ---
def process_step(env, wafer, step_name, resource):
    """
    Simulates a wafer going through a specific fabrication step.
    This is a SimPy process function, meaning it yields control back to the
    environment when it waits for something (like a resource or a timeout).

    :param env: The SimPy environment.
    :param wafer: The Wafer object currently being processed.
    :param step_name: The name of the current fabrication step (e.g., "Lithography").
    :param resource: The SimPy Resource object representing the machines at this step.
    """
    # 1. Record the time the wafer enters the queue for this specific step.
    wafer.queue_enter_times[step_name] = env.now

    # 2. Request a machine (resource) from the pool.
    #    'with resource.request() as req:' automatically handles requesting
    #    and releasing the resource.
    #    'yield req' pauses this wafer's process until a machine becomes available.
    with resource.request() as req:
        yield req # Wafer waits here until an available machine in the step is acquired.

        # 3. Once a machine is acquired, calculate the time spent waiting in the queue.
        queue_time = env.now - wafer.queue_enter_times[step_name]
        step_queue_times[step_name].append(queue_time) # Store queue time for later analysis.

        # 4. Record the precise simulation time when processing actually begins.
        wafer.start_times[step_name] = env.now

        # 5. Determine the actual processing time for this wafer at this step.
        #    We use a Gaussian (normal) distribution to add realistic variability.
        #    'max(1, ...)' ensures the processing time is never less than 1 unit,
        #    preventing non-physical negative or zero times.
        process_time_mean = FAB_STEPS_CONFIG[step_name]["process_time_mean"]
        process_time_std = FAB_STEPS_CONFIG[step_name]["process_time_std"]
        actual_process_time = max(1, random.gauss(process_time_mean, process_time_std))

        # 6. Simulate the passage of time for the actual processing.
        #    'yield env.timeout(duration)' pauses the current process for 'duration'
        #    simulation time units, allowing other processes to run.
        yield env.timeout(actual_process_time)

        # 7. Record the time the processing is completed for this step.
        wafer.end_times[step_name] = env.now
        step_process_times[step_name].append(actual_process_time) # Store actual process time.

        # 8. Record a snapshot of resource utilization at the moment this wafer finishes.
        #    resource.capacity is total machines. resource.count is available machines.
        #    So (resource.capacity - resource.count) is the number of busy machines.
        busy_machines = resource.capacity - resource.count
        busy_ratio = busy_machines / resource.capacity if resource.capacity > 0 else 0
        resource_utilization_data[step_name].append((env.now, busy_ratio))

        # Print a message indicating completion of this step.
        print(f"[{env.now:.2f}] {wafer.name} finished {step_name}.")


# --- Wafer Generator Process ---
def wafer_generator(env, fab_steps):
    """
    Generates new Wafer objects at random intervals and initiates their lifecycle
    through the fabrication process. This is a continuous process that runs
    for the duration of the simulation.

    :param env: The SimPy environment.
    :param fab_steps: A dictionary mapping step names to their SimPy Resource objects.
    """
    wafer_count = 0 # Counter for naming wafers uniquely
    while True: # This loop runs indefinitely until the main simulation timer expires.
        # 1. Wait for a random period before creating the next wafer.
        #    random.expovariate is used for inter-arrival times, common in queuing models.
        yield env.timeout(random.expovariate(1.0 / WAFER_ARRIVAL_INTERVAL))

        # 2. Increment the count and create a new Wafer object.
        wafer_count += 1
        wafer = Wafer(env, f"Wafer_{wafer_count}")
        print(f"[{env.now:.2f}] {wafer.name} arrived.")

        # 3. Start a new SimPy process for this wafer's entire lifecycle.
        #    Each wafer's journey runs concurrently.
        env.process(wafer_lifecycle(env, wafer, fab_steps))


# --- Wafer Lifecycle Process ---
def wafer_lifecycle(env, wafer, fab_steps):
    """
    Defines the sequential flow of a single wafer through all predefined
    fabrication steps. The wafer must complete one step before moving to the next.

    :param env: The SimPy environment.
    :param wafer: The Wafer object undergoing the process.
    :param fab_steps: A dictionary mapping step names to their SimPy Resource objects.
    """
    # Iterate through each step in the order defined in FAB_STEPS_CONFIG.
    for step_name, resource in fab_steps.items():
        # 'yield env.process(process_step(...))' means this wafer's lifecycle
        # process will pause and wait for the 'process_step' function
        # to fully complete before proceeding to the next step in the loop.
        yield env.process(process_step(env, wafer, step_name, resource))

    # Once all steps are finished, calculate the total time the wafer spent in the system.
    cycle_time = env.now - wafer.arrival_time
    wafer_cycle_times.append(cycle_time) # Add to the global list for overall statistics.
    print(f"[{env.now:.2f}] {wafer.name} completed all steps. Cycle time: {cycle_time:.2f}")


# --- Main Simulation Setup and Run ---
def run_simulation():
    """
    Sets up the SimPy environment, initializes resources, starts the
    wafer generation, runs the simulation, and then prints the aggregated results.
    """
    print("--- Starting Wafer-Level Process Flow Simulation ---")

    # 1. Create the SimPy simulation environment. This is the central object
    #    that manages time and processes.
    env = simpy.Environment()

    # 2. Initialize SimPy Resource objects for each fabrication step.
    #    Each resource represents a pool of machines at that step, with a given capacity.
    fab_resources = {
        step_name: simpy.Resource(env, capacity=config["capacity"])
        for step_name, config in FAB_STEPS_CONFIG.items()
    }

    # 3. Start the wafer_generator process. This is the source of new wafers.
    #    It's an ongoing process running in the background.
    env.process(wafer_generator(env, fab_resources))

    # 4. Run the simulation. The environment will process events until
    #    'SIMULATION_TIME' is reached.
    env.run(until=SIMULATION_TIME)

    print("\n--- Simulation Complete ---")
    print("\n--- Simulation Results ---")

    # --- Analysis and Reporting ---
    # 5. Report overall wafer completion statistics.
    if wafer_cycle_times: # Check if any wafers successfully completed all steps.
        print(f"\nTotal wafers completed: {len(wafer_cycle_times)}")
        print(f"Average Wafer Cycle Time: {statistics.mean(wafer_cycle_times):.2f} units")
        # Only calculate standard deviation if there's enough data (more than one wafer).
        if len(wafer_cycle_times) > 1:
            print(f"Std Dev Wafer Cycle Time: {statistics.stdev(wafer_cycle_times):.2f} units")
    else:
        print("\nNo wafers completed during simulation.")


    print("\n--- Step Performance Metrics ---")
    # 6. Report detailed performance metrics for each individual fabrication step.
    for step_name, config in FAB_STEPS_CONFIG.items():
        print(f"\nStep: {step_name} (Capacity: {config['capacity']})")

        # Report Average Queue Time for this step.
        if step_queue_times[step_name]:
            avg_queue = statistics.mean(step_queue_times[step_name])
            print(f"  Average Queue Time: {avg_queue:.2f} units")
            # Report Max Queue Time if there's more than one data point.
            if len(step_queue_times[step_name]) > 1:
                print(f"  Max Queue Time: {max(step_queue_times[step_name]):.2f} units")
        else:
            print("  No queue data available (no wafers processed or waited at this step).")

        # Report Average Process Time for this step.
        if step_process_times[step_name]:
            avg_process = statistics.mean(step_process_times[step_name])
            print(f"  Average Process Time: {avg_process:.2f} units (Configured Mean: {config['process_time_mean']})")
        else:
            print("  No process data available.")

        # Calculate and report Estimated Average Resource Utilization.
        # This approximates the area under the busy-resource-over-time curve.
        total_busy_hours = 0
        previous_time = 0
        previous_busy_ratio = 0

        # Sort the utilization snapshots by time to ensure correct interval calculation.
        sorted_util_data = sorted(resource_utilization_data[step_name], key=lambda x: x[0])

        if sorted_util_data:
            # Initialize with the first data point's time and busy ratio.
            previous_time = sorted_util_data[0][0]
            previous_busy_ratio = sorted_util_data[0][1]

            # Iterate through the rest of the sorted data points.
            for current_time, current_busy_ratio in sorted_util_data[1:]:
                # Calculate the duration of the previous busy ratio.
                duration = current_time - previous_time
                # Add the 'busy time' (ratio * capacity * duration) to total.
                total_busy_hours += previous_busy_ratio * config['capacity'] * duration
                # Update for the next interval.
                previous_time = current_time
                previous_busy_ratio = current_busy_ratio

            # Account for the time from the last recorded event until the simulation ends.
            duration_to_end = SIMULATION_TIME - previous_time
            if duration_to_end > 0:
                total_busy_hours += previous_busy_ratio * config['capacity'] * duration_to_end

        # Calculate the overall average utilization percentage.
        if SIMULATION_TIME > 0 and config['capacity'] > 0:
            total_possible_busy_hours = SIMULATION_TIME * config['capacity']
            if total_possible_busy_hours > 0:
                average_utilization_percentage = (total_busy_hours / total_possible_busy_hours) * 100
                print(f"  Estimated Average Resource Utilization: {average_utilization_percentage:.2f}%")
            else:
                print("  Resource utilization not calculable (Total possible busy hours is zero).")
        else:
            print("  Resource utilization not calculable (Simulation time or capacity is zero).")


# This standard Python construct ensures that run_simulation() is called only
# when this script is executed directly, not when it's imported as a module into another script.
if __name__ == "__main__":
    run_simulation()


--- Starting Wafer-Level Process Flow Simulation ---
[0.65] Wafer_1 arrived.
[6.10] Wafer_1 finished Lithography.
[8.49] Wafer_2 arrived.
[9.70] Wafer_3 arrived.
[11.09] Wafer_4 arrived.
[13.56] Wafer_1 finished Etch.
[17.36] Wafer_2 finished Lithography.
[17.73] Wafer_3 finished Lithography.
[21.32] Wafer_1 finished Deposition.
[23.02] Wafer_5 arrived.
[24.02] Wafer_4 finished Lithography.
[25.29] Wafer_6 arrived.
[25.98] Wafer_3 finished Etch.
[26.73] Wafer_2 finished Etch.
[27.18] Wafer_7 arrived.
[32.20] Wafer_4 finished Etch.
[34.16] Wafer_5 finished Lithography.
[34.93] Wafer_1 finished IonImplant.
[35.17] Wafer_2 finished Deposition.
[37.32] Wafer_6 finished Lithography.
[38.45] Wafer_1 finished Metrology.
[38.45] Wafer_1 completed all steps. Cycle time: 37.80
[38.47] Wafer_8 arrived.
[40.45] Wafer_3 finished Deposition.
[40.46] Wafer_9 arrived.
[41.44] Wafer_10 arrived.
[43.53] Wafer_11 arrived.
[44.81] Wafer_5 finished Etch.
[45.40] Wafer_6 finished Etch.
[45.68] Wafer_4 finis